In [70]:
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
    

In [71]:
#data cleaning

face_detect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')   #face cascade
eyes_detect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')                  #eye cascade



if face_detect.empty():
    raise IOError("Failed to load haarcascade_frontalface_default.xml")

if eyes_detect.empty():
    raise IOError("Failed to load haarcascade_eye.xml")


In [72]:
def cropped_image_with_2eyes(image_path):
    img=cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to read image at {image_path}")
        return None
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_detect.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        roi_gray=gray[y:y+h, x:x+w]
        roi_color=img[y:y+h,x:x+w]
        eyes=eyes_detect.detectMultiScale(roi_gray)
        if len(eyes)>=2:
            return roi_color
    return None

In [73]:
images_path='./images'
cropped_images='./cropped_images'


In [74]:
import os
img_dirs=[]
for entry in os.scandir(images_path):
    if entry.is_dir():
        img_dirs.append(entry.path)

In [75]:
img_dirs

['./images\\elon_musk',
 './images\\jeff_bezos',
 './images\\pavel_durov',
 './images\\sam_altman',
 './images\\satya_nandella',
 './images\\sundar_pichai']

In [76]:
import shutil
if os.path.exists(cropped_images):
    shutil.rmtree(cropped_images)
os.makedirs(cropped_images)


In [77]:
cropped_images_dirs = []
name_dict = {}

for img_dir in img_dirs:
    name = img_dir.split('/')[-1]
    print(name)

    for entry in os.scandir(img_dir):
        roi_color = cropped_image_with_2eyes(entry.path)
        if roi_color is not None:
            # Create directory if not exists
            cropped_folder = os.path.join(cropped_images, name)
            os.makedirs(cropped_folder, exist_ok=True)

            # Create unique image path
            cropped_image_path = os.path.join(cropped_folder, entry.name)

            # Save image
            cv2.imwrite(cropped_image_path, roi_color)

            # Track image path and name
            cropped_images_dirs.append(cropped_image_path)
            name_dict[cropped_image_path] = name

    print(f'Processed {name} images')

images\elon_musk
Error: Unable to read image at ./images\elon_musk\04dtmd6g_elon-musk_625x300_22_November_24.jpg
Error: Unable to read image at ./images\elon_musk\1574363408-796.jpg
Error: Unable to read image at ./images\elon_musk\20250414030810_elonmuskssn.jpg
Error: Unable to read image at ./images\elon_musk\3VNRE7RXQDEWEFFS23ZEBGIZ2Q.jpg
Error: Unable to read image at ./images\elon_musk\679ae348eb49a-elon-musk-302611153-16x9.jpg
Error: Unable to read image at ./images\elon_musk\6EHE75VUFNNGJEM67MY6K3KBII.jpg
Error: Unable to read image at ./images\elon_musk\elon-musk-1-gty-gmh-250604_1749042286752_hpMain.jpg
Error: Unable to read image at ./images\elon_musk\elon-musk-1-rt-gmh-250528_1748436299357_hpMain.jpg
Error: Unable to read image at ./images\elon_musk\https%3A%2F%2Fcloudfront-us-east-2.images.arcpublishing.com%2Freuters%2FNPYZQCXOD5NHFDJXBOQLVZD7PM.jpg
Error: Unable to read image at ./images\elon_musk\spacex-founder-elon-musk-171701448-16x9_0.jpg
Error: Unable to read image at

In [79]:
#using wavelet transform to extract features
import pywt
def wavelet_transform(image_path,mode='haar',level=2):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    array=img
    array=cv2.cvtColor(array, cv2.COLOR_BGR2GRAY)
    array=np.float32(array)
    array /= 255.0  # Normalize to [0, 1]
    coeffs = pywt.wavedec2(array, 'haar', level=level)
    coeffs_H=list(coeffs)
    coeffs_H[0] *=0  # Set the approximation coefficients to zero

    array_H = pywt.waverec2(coeffs_H, 'haar')
    array_H *= 255.0  # Scale back to [0, 255]
    array_H=np.uint8(array_H)  # Convert to uint8

    return array_H

In [81]:
class_dict = {}
count = 0
for namee in name_dict.values():
    if namee not in class_dict:
        class_dict[namee] = count
        count += 1
class_dict

{'images\\elon_musk': 0,
 'images\\jeff_bezos': 1,
 'images\\pavel_durov': 2,
 'images\\sam_altman': 3,
 'images\\satya_nandella': 4,
 'images\\sundar_pichai': 5}

In [ ]:
X=[]
Y=[]

for namee, training_files in name_dict.items():
    for training_image in training_files:
        img=cv2.imread(training_image)
        if img is None:
            continue
        scalled_raw_image = cv2.resize(img, (32,32))  # Resize to 32x32
        img_har= wavelet_transform(training_image, 'db1', 5)
        scalled_raw_har = cv2.resize(img_har, (32,32))
        combined_image = np.hstack((scalled_raw_image, scalled_raw_har))
        X.append(combined_image)
        Y.append(class_dict[namee])


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
